Выполнять проектную работу буду на примере готового размеченного датасета со спутниковыми снимками:
https://project.inria.fr/aerialimagelabeling/

Для решения задачи семантической сегментации одного класса («площадь дома») на изображениях большого размера (5000×5000 пикселей) воспользуемся архитектурой U-Net — популярной архитектурой для сегментации, хорошо работающую со спутниковыми изображениями.
В рамках работы пробовал ResNet18 и ResNet34. ResNet34 очень долго считает по времени без существенного "выйгрыша" по метрикам. Поэтому с целью ускорения вычислений и уменьшения платы яндекесу, но без существенной потери качества, итогововая модель рассчитана на ResNet18.

Поскольку исходные изображения очень большие (5000×5000), прямая подача их в сеть невозможна из-за ограничений видеопамяти (арендую T4 16 GPU от Яндекса). Поэтому будем использовать тайлинг (разделение изображения на патчи), 512×512 с перекрытием (в рамках работы даннный параметр был отдельно подобран от 256 до 1024).

Модель обучается на тайлах 512×512, что позволяет обрабатывать изображения любого размера.
Маска в формате градаций серого, где 255 = дом, 0 = фон.
Используется бинарная сегментация (num_classes=1 + BCEWithLogitsLoss).


Стандартная модель U-Net

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import cv2
import numpy as np
import os
import glob
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
import torch
# Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")
if device.type == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

Используемое устройство: cuda
   GPU: NVIDIA L4


In [8]:

# ----------------------------
# 1. Определение модели (U-Net на базе ResNet18 backbone)
# ----------------------------

from torchvision.models import resnet18, ResNet18_Weights

class UNetResNet18(nn.Module):
    def __init__(self, num_classes=1):
        super().__init__()
        weights = ResNet18_Weights.IMAGENET1K_V1
        encoder = resnet18(weights=weights)

        # Сохраняем все нужные уровни
        self.enc0 = nn.Sequential(encoder.conv1, encoder.bn1, encoder.relu)  # до maxpool → H/2
        self.pool = encoder.maxpool  # → H/4
        self.enc1 = encoder.layer1   # → H/4
        self.enc2 = encoder.layer2   # → H/8
        self.enc3 = encoder.layer3   # → H/16
        self.enc4 = encoder.layer4   # → H/32

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)  # H/16
        self.decoder4 = self._conv_block(512, 256)

        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)  # H/8
        self.decoder3 = self._conv_block(256, 128)

        self.upconv2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)   # H/4
        self.decoder2 = self._conv_block(128, 64)

        self.upconv1 = nn.ConvTranspose2d(64, 64, kernel_size=2, stride=2)    # H/2
        self.decoder1 = self._conv_block(128, 64)

        self.upconv0 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)    # H
        self.decoder0 = self._conv_block(32, 32)  # enc0 имеет 64 канала, но мы используем только часть

        self.final = nn.Conv2d(32, num_classes, kernel_size=1)

    def _conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        # Encoder
        e0 = self.enc0(x)       # H/2, 64
        x = self.pool(e0)       # H/4
        e1 = self.enc1(x)       # H/4, 64
        e2 = self.enc2(e1)      # H/8, 128
        e3 = self.enc3(e2)      # H/16, 256
        e4 = self.enc4(e3)      # H/32, 512

        # Decoder
        d4 = self.upconv4(e4)               # H/16, 256
        d4 = torch.cat([d4, e3], dim=1)     # 256+256 = 512
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)               # H/8, 128
        d3 = torch.cat([d3, e2], dim=1)     # 128+128 = 256
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)               # H/4, 64
        d2 = torch.cat([d2, e1], dim=1)     # 64+64 = 128
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)               # H/2, 64
        d1 = torch.cat([d1, e0], dim=1)     # 64+64 = 128 ← теперь размеры совпадают!
        d1 = self.decoder1(d1)

        d0 = self.upconv0(d1)               # H, 32
        d0 = self.decoder0(d0)

        out = self.final(d0)
        return out
    
# ----------------------------
# 2. Загрузчик данных (разбивает большие изображения на тайлы)
# ----------------------------

class LargeImageDataset(Dataset):
    def __init__(self, image_paths, mask_paths, tile_size=512):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.tile_size = tile_size
        self.tiles = self._build_tile_list()

    def _build_tile_list(self):
        tiles = []
        for i, (img_path, mask_path) in enumerate(zip(self.image_paths, self.mask_paths)):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                raise IOError(f"Не удалось загрузить маску: {mask_path}")
            h, w = mask.shape
            for y in range(0, h, self.tile_size):
                for x in range(0, w, self.tile_size):
                    tiles.append((i, y, x))
        return tiles

    def __len__(self):
        return len(self.tiles)

    def __getitem__(self, idx):
        img_idx, y, x = self.tiles[idx]
        # Читаем изображение и маску КАЖДЫЙ РАЗ
        img = cv2.imread(self.image_paths[img_idx])
        mask = cv2.imread(self.mask_paths[img_idx], cv2.IMREAD_GRAYSCALE)
        
        if img is None or mask is None:
            raise IOError(f"Ошибка чтения: {self.image_paths[img_idx]} или маска")

        h, w = img.shape[:2]
        x_end = min(x + self.tile_size, w)
        y_end = min(y + self.tile_size, h)

        tile_img = img[y:y_end, x:x_end]
        tile_mask = mask[y:y_end, x:x_end]

        pad_h = self.tile_size - tile_img.shape[0]
        pad_w = self.tile_size - tile_img.shape[1]
        tile_img = np.pad(tile_img, ((0, pad_h), (0, pad_w), (0, 0)), mode='constant')
        tile_mask = np.pad(tile_mask, ((0, pad_h), (0, pad_w)), mode='constant')

        tile_img = torch.from_numpy(tile_img).permute(2, 0, 1).float() / 255.0
        tile_mask = torch.from_numpy(tile_mask).unsqueeze(0).float() / 255.0

        return tile_img, tile_mask
    
# ----------------------------
# 3. Обучение
# ----------------------------
import time
from datetime import datetime

def calculate_iou(pred, target, threshold=0.5):
    """
    Вычисляет IoU для бинарной сегментации.
    pred: Tensor [B, 1, H, W] — логиты или вероятности
    target: Tensor [B, 1, H, W] — маски (0 или 1)
    """
    with torch.no_grad():
        pred = torch.sigmoid(pred)  # преобразуем логиты в вероятности
        pred = (pred > threshold).float()
        target = (target > 0.5).float()  # на случай, если маска не бинарная

        intersection = (pred * target).sum(dim=(1, 2, 3))
        union = (pred + target - pred * target).sum(dim=(1, 2, 3))
        iou = (intersection + 1e-6) / (union + 1e-6)  # eps для стабильности
        return iou.mean().item()

def calculate_accuracy(pred, target, threshold=0.5):
    """Вычисляет точность (pixel accuracy)"""
    with torch.no_grad():
        pred = torch.sigmoid(pred)
        pred = (pred > threshold).float()
        target = (target > 0.5).float()
        correct = (pred == target).float().sum()
        total = target.numel()
        return (correct / total).item()

def calculate_iou_gpu(pred, target, threshold=0.5):
    with torch.no_grad():
        pred = (torch.sigmoid(pred) > threshold).float()
        target = (target > 0.5).float()
        intersection = (pred * target).sum(dim=(1, 2, 3))
        union = (pred + target - pred * target).sum(dim=(1, 2, 3))
        iou = (intersection + 1e-6) / (union + 1e-6)
        return iou.sum()  # возвращаем сумму, а не среднее

def calculate_accuracy_gpu(pred, target, threshold=0.5):
    with torch.no_grad():
        pred = (torch.sigmoid(pred) > threshold).float()
        target = (target > 0.5).float()
        correct = (pred == target).float().sum()
        return correct  # возвращаем число правильных пикселей
    
def train_model():
    print("=" * 60)
    print(f"Запуск обучения модели: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 60)

    image_dir = "Data/train/images"
    mask_dir = "Data/train/masks"

    print(f"Папка с изображениями: {os.path.abspath(image_dir)}")
    print(f"Папка с масками:       {os.path.abspath(mask_dir)}")

    image_extensions = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
    mask_extensions = {".png", ".jpg", ".jpeg", ".tif", ".tiff"}

    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(image_dir, f"*{ext}")))
        image_files.extend(glob.glob(os.path.join(image_dir, f"*{ext.upper()}")))
    image_files = sorted(list(set(image_files)))

    if not image_files:
        raise FileNotFoundError(f"Не найдено изображений в {image_dir}")

    print(f" Найдено изображений: {len(image_files)}")

    image_paths = []
    mask_paths = []
    for img_path in image_files:
        img_name = Path(img_path).stem
        mask_found = False
        for ext in mask_extensions:
            for candidate in [ext, ext.upper()]:
                mask_path = os.path.join(mask_dir, img_name + candidate)
                if os.path.exists(mask_path):
                    image_paths.append(img_path)
                    mask_paths.append(mask_path)
                    mask_found = True
                    break
            if mask_found:
                break
        if not mask_found:
            print(f" Пропущено (маска не найдена): {os.path.basename(img_path)}")

    if not image_paths:
        raise FileNotFoundError("Не найдено ни одной пары изображение/маска")

    print(f"Найдено пар изображение/маска: {len(image_paths)}")

    X_train, X_val, y_train, y_val = train_test_split(
        image_paths, mask_paths, test_size=0.2, random_state=42
    )
    print(f"Обучающих примеров: {len(X_train)}")
    print(f"Валидационных примеров: {len(X_val)}")

    print("Создание датасетов...")
    train_dataset = LargeImageDataset(X_train, y_train, tile_size=512)
    val_dataset = LargeImageDataset(X_val, y_val, tile_size=512)
    print(f"Общее количество тайлов (train): {len(train_dataset)}")
    print(f"Общее количество тайлов (val):   {len(val_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=14, shuffle=True, num_workers=8, pin_memory=True, prefetch_factor=2, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=14, shuffle=False, num_workers=8, pin_memory=True, prefetch_factor=2, persistent_workers=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f" Используемое устройство: {device}")
    if device.type == "cuda":
        print(f"   GPU: {torch.cuda.get_device_name(0)}")

    print("Инициализация модели U-Net с ResNet14 backbone...")
    model = UNetResNet18(num_classes=1).to(device)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"   Всего параметров: {total_params:,}")
    print(f"   Обучаемых параметров: {trainable_params:,}")

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    print("\nНачало обучения...\n")

    # --- Основной цикл обучения с валидацией ---
    
    # --- Профилирование батчей ---
    data_loading_times = []
    gpu_compute_times = []
    log_every = 50  # считать метрики каждые 20 батчей
    
    scaler = torch.cuda.amp.GradScaler()
    for epoch in range(4):
        epoch_start = time.time()

        # ===== Обучение =====
        model.train()
        train_loss = 0.0
        train_iou = 0.0
        train_acc = 0.0
        batch_idx = 0
        num_metric_batches = 0
        for images, masks in train_loader:
            
            batch_idx = batch_idx + 1
            # Время загрузки данных (CPU)
            start_data = time.time()
            images, masks = images.to(device, non_blocking=True), masks.to(device, non_blocking=True)
            torch.cuda.synchronize()  # ждём, пока данные попадут на GPU
            data_time = time.time() - start_data
            
            # Время вычислений на GPU
            start_gpu = time.time()
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, masks)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
           
            torch.cuda.synchronize()  # синхронизация GPU     
            gpu_time = time.time() - start_gpu
    
            train_loss += loss.item()
            
            data_loading_times.append(data_time)
            gpu_compute_times.append(gpu_time)

            if batch_idx  % log_every == 0:
                train_iou += calculate_iou(outputs, masks)
                train_acc += calculate_accuracy(outputs, masks)
                num_metric_batches += 1
                print(f"Batch {batch_idx+1}: "
                      f"Data loading: {data_time:.3f}s | "
                      f"GPU compute: {gpu_time:.3f}s | "
                      f"train_iou: {train_iou / num_metric_batches:.3f} | "
                      f"train_acc: {train_acc / num_metric_batches:.3f} | "
                     )            

        avg_train_loss = train_loss / len(train_loader)
        avg_train_iou = train_iou / num_metric_batches
        avg_train_acc = train_acc / num_metric_batches
        
        avg_data = sum(data_loading_times) / len(data_loading_times)
        avg_gpu = sum(gpu_compute_times) / len(gpu_compute_times)

        print(f"\n Среднее за батч:")
        print(f"   Загрузка данных: {avg_data:.3f} сек")
        print(f"   Вычисления на GPU: {avg_gpu:.3f} сек")
        print(f"   GPU utilization: ~{avg_gpu / (avg_data + avg_gpu) * 100:.1f}%")

        if avg_data > avg_gpu:
            print("УЗКОЕ МЕСТО: загрузка данных (CPU/DataLoader)")
        else:
            print("GPU — основное время")        

        # ===== Валидация =====
        model.eval()
        val_loss = 0.0
        val_iou = 0.0
        val_acc = 0.0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)

                val_loss += loss.item()
                val_iou += calculate_iou(outputs, masks)
                val_acc += calculate_accuracy(outputs, masks)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_iou = val_iou / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)

        epoch_time = time.time() - epoch_start
        print(f"Epoch {epoch+1:02d}/{10} | "
              f"Train Loss: {avg_train_loss:.4f} | Train IoU: {avg_train_iou:.4f} | Train Acc: {avg_train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | Val IoU: {avg_val_iou:.4f} | Val Acc: {avg_val_acc:.4f} | "
              f"Time: {epoch_time:.1f}s")

    # Сохранение модели
    model_path = "house_segmentation_model.pth"
    torch.save(model.state_dict(), model_path)
    print(f"\nМодель успешно сохранена: {os.path.abspath(model_path)}")
    print("=" * 60)

# ----------------------------
# 4. Инференс большой картинки (для Streamlit)
# ----------------------------

def predict_large_image(model_path, image_path, tile_size=512, device="cpu"):
    model = UNetResNet18(num_classes=1)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval().to(device)

    img = cv2.imread(image_path)
    h, w = img.shape[:2]
    pred_mask = np.zeros((h, w), dtype=np.float32)

    with torch.no_grad():
        for y in range(0, h, tile_size):
            for x in range(0, w, tile_size):
                y_end = min(y + tile_size, h)
                x_end = min(x + tile_size, w)

                tile = img[y:y_end, x:x_end]
                pad_h = tile_size - tile.shape[0]
                pad_w = tile_size - tile.shape[1]
                tile_padded = np.pad(tile, ((0, pad_h), (0, pad_w), (0, 0)), mode='constant')
                tile_tensor = torch.from_numpy(tile_padded).permute(2, 0, 1).float() / 255.0
                tile_tensor = tile_tensor.unsqueeze(0).to(device)

                output = model(tile_tensor)
                prob = torch.sigmoid(output).cpu().numpy()[0, 0]
                prob = prob[:y_end - y, :x_end - x]
                pred_mask[y:y_end, x:x_end] = prob

    return (pred_mask > 0.5).astype(np.uint8) * 255  # Бинарная маска

# ----------------------------
# Запуск обучения (при необходимости)
# ----------------------------

if __name__ == "__main__":
    train_model()  

Запуск обучения модели: 2026-01-09 14:20:31
Папка с изображениями: /home/jupyter/project/Data/train/images
Папка с масками:       /home/jupyter/project/Data/train/masks
 Найдено изображений: 180
Найдено пар изображение/маска: 180
Обучающих примеров: 144
Валидационных примеров: 36
Создание датасетов...
Общее количество тайлов (train): 14400
Общее количество тайлов (val):   3600
 Используемое устройство: cuda
   GPU: NVIDIA L4
Инициализация модели U-Net с ResNet14 backbone...
   Всего параметров: 14,342,337
   Обучаемых параметров: 14,342,337

Начало обучения...

Batch 51: Data loading: 0.005s | GPU compute: 0.193s | train_iou: 0.143 | train_acc: 0.875 | 
Batch 101: Data loading: 0.005s | GPU compute: 0.193s | train_iou: 0.143 | train_acc: 0.835 | 
Batch 151: Data loading: 0.004s | GPU compute: 0.193s | train_iou: 0.119 | train_acc: 0.811 | 
Batch 201: Data loading: 0.004s | GPU compute: 0.198s | train_iou: 0.236 | train_acc: 0.838 | 
Batch 251: Data loading: 0.005s | GPU compute: 0.193s

Обучение проводилось в 4 эпохи и заняло около 3-х часов работы процессорного и GPU времени.
Результаты получились удовлетворительные, в целом коррелирующие с лидербордом по данному датасету (https://project.inria.fr/aerialimagelabeling/leaderboard/)
Val IoU: 0.7515 | Val Acc: 0.9619Val 

Рассчитанная модель сохранена в отдельный файл для дальнейшего использования в решении задачи.